In [ ]:
import os
import numpy as np
import random
import time
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
from geodesic_kmeans import constrained_geodesic_kmeans
from potential_fields import divide_regions_potential_fields
from wavefront_division import divide_regions_wavefront
# Используем нашу новую функцию визуализации с plotly
from visualization import visualize_regions
from comparison import compare_methods

def test_region_division(map_size=(30, 30, 10), num_agents=4, obstacle_type='maze', 
                      visualize=True, save_results=True):
    """
    Тестирует алгоритмы разделения регионов на различных типах карт с препятствиями
    
    Параметры:
        map_size: размер карты (width, height, depth)
        num_agents: количество агентов
        obstacle_type: тип препятствий ('maze', 'mountains', 'spheres')
        visualize: флаг для отображения результатов
        save_results: флаг для сохранения результатов
    """
    # Импортируем генераторы препятствий
    from obstacles.maze_generator import generate_maze, render_maze
    from obstacles.mountain_generator import generate_mountains
    from obstacles.spherical_obstacles_generator import generate_spherical_obstacles
    
    width, height, depth = map_size
    
    # Создаем директорию для результатов
    if save_results:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_dir = f"results_{obstacle_type}_{timestamp}"
        os.makedirs(results_dir, exist_ok=True)
    else:
        results_dir = None
    
    # Генерируем карту с препятствиями в зависимости от выбранного типа
    if obstacle_type == 'maze':
        print("Генерация лабиринта...")
        rows, cols = width // 3, height // 3
        removed_walls = generate_maze(rows, cols, add_connections=5)
        map_3d = render_maze(removed_walls, rows, cols, 
                           passage_width=2, wall_width=1, map_height=depth)
    
    elif obstacle_type == 'mountains':
        print("Генерация горного ландшафта...")
        map_3d = generate_mountains(num_mountains=5, map_size=(width, height, depth))
    
    elif obstacle_type == 'spheres':
        print("Генерация сферических препятствий...")
        map_3d = generate_spherical_obstacles(
            width, height, depth, num_obstacles=15, 
            radius_range=(2, 5), wall_count=3, wall_length=8
        )
    else:
        raise ValueError(f"Неизвестный тип препятствий: {obstacle_type}")
    
    print(f"Карта создана. Размер: {map_3d.shape}")
    
    # Генерируем случайные позиции для агентов в свободных ячейках
    free_cells = np.argwhere(map_3d == 0)
    if len(free_cells) < num_agents:
        raise ValueError(f"Недостаточно свободных ячеек для {num_agents} агентов!")
    
    agent_indices = random.sample(range(len(free_cells)), num_agents)
    agent_positions = []
    
    for idx in agent_indices:
        z, y, x = free_cells[idx]
        agent_positions.append([x, y, z])  # Формат [x, y, z]
    
    print(f"Позиции агентов: {agent_positions}")
    
    # Запускаем сравнение всех методов
    methods = [
        ("Геодезический K-means", constrained_geodesic_kmeans),
        ("Потенциальные поля", divide_regions_potential_fields),
        ("Волновой алгоритм", divide_regions_wavefront)
    ]
    
    # Создаем директории для каждого метода
    if save_results:
        for method_name, _ in methods:
            method_dir = f"{results_dir}/{method_name.lower().replace(' ', '_')}"
            os.makedirs(method_dir, exist_ok=True)
    
    # Запускаем каждый метод и сохраняем результат
    results = {}
    for method_name, method_func in methods:
        print(f"\n===== Тестирование метода: {method_name} =====")
        
        # Засекаем время выполнения
        start_time = time.time()
        result = method_func(map_3d, num_agents, agent_positions)
        execution_time = time.time() - start_time
        
        results[method_name] = {
            "region_assignment": result,
            "execution_time": execution_time,
            "region_sizes": [np.sum(result == i) for i in range(num_agents)]
        }
        
        # Визуализируем результат с использованием plotly
        if visualize:
            save_path = f"{results_dir}/{method_name.lower().replace(' ', '_')}/result.png" if save_results else None
            fig = visualize_regions(map_3d, result, agent_positions, method_name, save_path)
            fig.show()  # Явно показываем фигуру plotly
    
    # Создаем сравнительную таблицу
    comparison_table = {}
    comparison_table["Метод"] = []
    comparison_table["Время выполнения (сек)"] = []
    comparison_table["Средний размер региона"] = []
    comparison_table["Минимальный размер"] = []
    comparison_table["Максимальный размер"] = []
    comparison_table["Дисбаланс (%)"] = []
    
    for method_name, data in results.items():
        region_sizes = data["region_sizes"]
        avg_size = sum(region_sizes) / len(region_sizes) if region_sizes else 0
        min_size = min(region_sizes) if region_sizes else 0
        max_size = max(region_sizes) if region_sizes else 0
        imbalance = (max_size - min_size) / avg_size * 100 if avg_size > 0 else 0
        
        comparison_table["Метод"].append(method_name)
        comparison_table["Время выполнения (сек)"].append(f"{data['execution_time']:.2f}")
        comparison_table["Средний размер региона"].append(f"{avg_size:.2f}")
        comparison_table["Минимальный размер"].append(str(min_size))
        comparison_table["Максимальный размер"].append(str(max_size))
        comparison_table["Дисбаланс (%)"].append(f"{imbalance:.2f}")
    
    # Создаем DataFrame и сохраняем
    df = pd.DataFrame(comparison_table)
    
    if save_results:
        df.to_csv(f"{results_dir}/comparison_results.csv", index=False)
    
    print("\n===== Результаты сравнения =====")
    print(df.to_string(index=False))
    
    # Создаем сравнительные графики с plotly
    if visualize:
        create_comparison_charts_plotly(results, obstacle_type, results_dir if save_results else None)
    
    return df

def create_comparison_charts_plotly(results, obstacle_type, save_dir=None):
    """Создает сравнительные графики для всех методов используя Plotly"""
    method_names = list(results.keys())
    execution_times = [results[m]["execution_time"] for m in method_names]
    
    # Вычисляем дисбаланс
    imbalances = []
    for method_name in method_names:
        region_sizes = results[method_name]["region_sizes"]
        avg_size = sum(region_sizes) / len(region_sizes) if region_sizes else 0
        min_size = min(region_sizes) if region_sizes else 0
        max_size = max(region_sizes) if region_sizes else 0
        imbalance = (max_size - min_size) / avg_size * 100 if avg_size > 0 else 0
        imbalances.append(imbalance)
    
    # Создаем подграфики
    fig = go.Figure()
    
    # Создаем вкладки для разных графиков
    fig = go.Figure()
    
    # График времени выполнения
    fig.add_trace(go.Bar(
        x=method_names,
        y=execution_times,
        name='Время выполнения',
        marker_color=['#3498db', '#e74c3c', '#2ecc71'],
        text=[f"{t:.2f}с" for t in execution_times],
        textposition='auto'
    ))
    
    # График дисбаланса
    fig.add_trace(go.Bar(
        x=method_names,
        y=imbalances,
        name='Дисбаланс',
        marker_color=['#9b59b6', '#f1c40f', '#34495e'],
        text=[f"{i:.2f}%" for i in imbalances],
        textposition='auto',
        visible=False
    ))
    
    # Добавляем кнопки для переключения между графиками
    fig.update_layout(
        title=f"Сравнение алгоритмов разделения регионов ({obstacle_type})",
        xaxis_title='Метод',
        yaxis_title='Значение',
        updatemenus=[
            dict(
                type="buttons",
                direction="right",
                active=0,
                x=0.57,
                y=1.2,
                buttons=list([
                    dict(
                        label="Время выполнения",
                        method="update",
                        args=[{"visible": [True, False]}, 
                             {"yaxis": {"title": "Время (сек)"}}]
                    ),
                    dict(
                        label="Дисбаланс",
                        method="update",
                        args=[{"visible": [False, True]}, 
                             {"yaxis": {"title": "Дисбаланс (%)"}}]
                    )
                ]),
            )
        ]
    )
    
    # Сохраняем график
    if save_dir:
        fig.write_image(f"{save_dir}/performance_comparison.png", scale=2)
    
    fig.show()

if __name__ == "__main__":
    # Устанавливаем seed для воспроизводимости результатов
    random.seed(42)
    np.random.seed(42)
    
    # Тестируем на разных типах препятствий
    obstacle_types = ['maze', 'mountains', 'spheres']
    
    for obstacle_type in obstacle_types:
        print(f"\n\n===== Тестирование на {obstacle_type} =====")
        test_region_division(
            map_size=(50, 50, 20),
            num_agents=5,
            obstacle_type=obstacle_type,
            visualize=True,
            save_results=True
        )